In [1]:
#from pyspark import SparkContext, SparkConf
from pyspark.rdd import RDD
from pyspark.sql import SparkSession
import numpy as np

In [2]:
spark = SparkSession.builder \
.master("local") \
.appName("mySparkSession") \
.config("k1", "vtest") \
.getOrCreate()

In [3]:
spark.catalog.clearCache()

In [4]:
spark._sc.version

u'2.3.4'

# DataFrame Creation

In [5]:
readings = spark.read.load("gs://saranv_bucket/Reducing Commercial Aviation Disaster/train.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

In [7]:
print type(readings), readings.count()

<class 'pyspark.sql.dataframe.DataFrame'> 4867421


In [8]:
readings.take(5)

[Row(crew=1, experiment=u'CA', time=0.01171875, seat=1, eeg_fp1=-5.28545, eeg_f7=26.775801, eeg_f8=-9.52731, eeg_t4=-12.7932, eeg_t6=16.7178, eeg_t5=33.737499, eeg_t3=23.712299, eeg_fp2=-6.6958699999999975, eeg_o1=29.2321, eeg_p3=24.842899, eeg_pz=3.92134, eeg_f3=18.447001, eeg_fz=1.07547, eeg_f4=3.09029, eeg_c4=37.368999, eeg_p4=17.437599, eeg_poz=19.2019, eeg_c3=20.5968, eeg_cz=-3.95115, eeg_o2=14.5076, ecg=-4520.0, r=817.705994, gsr=388.829987, event=u'A'),
 Row(crew=1, experiment=u'CA', time=0.015625, seat=1, eeg_fp1=-2.42842, eeg_f7=28.430901, eeg_f8=-9.32351, eeg_t4=-3.75723, eeg_t6=15.9693, eeg_t5=30.4436, eeg_t3=21.0103, eeg_fp2=-6.47472, eeg_o1=26.641399, eeg_p3=24.1388, eeg_pz=5.46991, eeg_f3=20.510599, eeg_fz=2.83354, eeg_f4=7.10177, eeg_c4=31.170799, eeg_p4=19.3997, eeg_poz=19.689501, eeg_c3=21.3547, eeg_cz=1.33212, eeg_o2=17.7502, ecg=-4520.0, r=817.705994, gsr=388.829987, event=u'A'),
 Row(crew=1, experiment=u'CA', time=0.01953125, seat=1, eeg_fp1=10.6715, eeg_f7=30.4202,

In [9]:
readings.printSchema()

root
 |-- crew: integer (nullable = true)
 |-- experiment: string (nullable = true)
 |-- time: double (nullable = true)
 |-- seat: integer (nullable = true)
 |-- eeg_fp1: double (nullable = true)
 |-- eeg_f7: double (nullable = true)
 |-- eeg_f8: double (nullable = true)
 |-- eeg_t4: double (nullable = true)
 |-- eeg_t6: double (nullable = true)
 |-- eeg_t5: double (nullable = true)
 |-- eeg_t3: double (nullable = true)
 |-- eeg_fp2: double (nullable = true)
 |-- eeg_o1: double (nullable = true)
 |-- eeg_p3: double (nullable = true)
 |-- eeg_pz: double (nullable = true)
 |-- eeg_f3: double (nullable = true)
 |-- eeg_fz: double (nullable = true)
 |-- eeg_f4: double (nullable = true)
 |-- eeg_c4: double (nullable = true)
 |-- eeg_p4: double (nullable = true)
 |-- eeg_poz: double (nullable = true)
 |-- eeg_c3: double (nullable = true)
 |-- eeg_cz: double (nullable = true)
 |-- eeg_o2: double (nullable = true)
 |-- ecg: double (nullable = true)
 |-- r: double (nullable = true)
 |-- gsr: do

In [10]:
readings.groupBy("experiment","event").agg({"crew":"count"}).alias("count").show()

+----------+-----+-----------+
|experiment|event|count(crew)|
+----------+-----+-----------+
|        DA|    D|     235329|
|        CA|    A|       5690|
|        DA|    A|    1423064|
|        SS|    A|    1420055|
|        CA|    C|    1652686|
|        SS|    B|     130597|
+----------+-----+-----------+



In [21]:
readings.groupBy("crew").agg({"crew":"count"}).alias("count").show()

+----+-----------+
|crew|count(crew)|
+----+-----------+
|   1|     447652|
|  13|     552724|
|   6|     552958|
|   3|     552795|
|   5|     552815|
|   4|     552881|
|   8|     549959|
|   7|     552769|
|   2|     552868|
+----+-----------+



In [11]:
readings.columns[4:-3]

['eeg_fp1',
 'eeg_f7',
 'eeg_f8',
 'eeg_t4',
 'eeg_t6',
 'eeg_t5',
 'eeg_t3',
 'eeg_fp2',
 'eeg_o1',
 'eeg_p3',
 'eeg_pz',
 'eeg_f3',
 'eeg_fz',
 'eeg_f4',
 'eeg_c4',
 'eeg_p4',
 'eeg_poz',
 'eeg_c3',
 'eeg_cz',
 'eeg_o2',
 'ecg']

In [6]:
#readings.selectExpr('percentile(eeg_fp1, 0.95)').show()
#pctl=readings.percentile(["eeg_fp1", "eeg_f7", "eeg_f8"], [0.03, 0.97])

pctls = {}
for col in readings.columns[4:-3]:
    pctls[col] = readings.approxQuantile(col,[0.015,0.985],0.01)
    print (col+"...",pctls[col][0],pctls[col][1])

('eeg_fp1...', -46.663898, 444.54599)
('eeg_f7...', -43.2048, 960.846008)
('eeg_f8...', -37.519001, 839.700989)
('eeg_t4...', -45.592201, 72.770798)
('eeg_t6...', -28.238199, 159.884003)
('eeg_t5...', -25.9058, 234.912994)
('eeg_t3...', -32.455101, 298.929993)
('eeg_fp2...', -52.695702, 868.7529910000003)
('eeg_o1...', -24.672701, 47.6105)
('eeg_p3...', -23.4737, 294.740997)
('eeg_pz...', -57.660999, 115.014)
('eeg_f3...', -62.4217, 654.7069700000002)
('eeg_fz...', -41.2272, 233.266998)
('eeg_f4...', -87.262199, 130.839996)
('eeg_c4...', -35.547001, 250.052994)
('eeg_p4...', -27.1192, 130.582993)
('eeg_poz...', -24.554701, 123.553001)
('eeg_c3...', -32.823898, 211.537003)
('eeg_cz...', -45.695, 241.632004)
('eeg_o2...', -23.494699, 192.660004)
('ecg...', -17872.0, 32939.800781)


In [7]:
readings_copy=readings
print ("Original Count = ", readings.count())
for col in readings.columns[4:-3]:
    readings_tmp=readings_copy.filter((readings[col]>pctls[col][0]) & (readings[col]<pctls[col][1]))
    readings_copy=readings_tmp
    print (col, readings_copy.count())

('Original Count = ', 4867421)
('eeg_fp1', 4731859)
('eeg_f7', 4673805)
('eeg_f8', 4580431)
('eeg_t4', 4527990)
('eeg_t6', 4487310)
('eeg_t5', 4432454)
('eeg_t3', 4395754)
('eeg_fp2', 4387951)
('eeg_o1', 4321155)
('eeg_p3', 4300185)
('eeg_pz', 4230473)
('eeg_f3', 4156009)
('eeg_fz', 4129504)
('eeg_f4', 4097960)
('eeg_c4', 4084093)
('eeg_p4', 4075618)
('eeg_poz', 4066582)
('eeg_c3', 4060157)
('eeg_cz', 4048006)
('eeg_o2', 4032699)
('ecg', 3886746)


In [8]:
spark.catalog.clearCache()

In [9]:
from pyspark.sql.functions import col
readings.createOrReplaceTempView("test_view")

In [ ]:
#readings_copy.rdd.saveAsPickleFile("/user/root/tmppickle")

In [ ]:
#!hdfs dfs -ls /user/root/tmppickle

In [12]:
#!hdfs dfs -cp /user/root/tmppickle "gs://saranv_bucket/Reducing Commercial Aviation Disaster/pickle"

In [13]:
#!gsutil ls "gs://saranv_bucket/Reducing Commercial Aviation Disaster/pickle"

In [8]:
#!hadoop fs -cp "gs://saranv_bucket/Reducing\ Commercial\ Aviation\ Disaster/pickle/" /user/root/
#pickleRdd = sc.pickleFile("/user/root/pickle").collect()
#readings_copy = spark.createDataFrame(pickleRdd)

In [9]:
#!hdfs dfs -ls /user/root/pickle

In [29]:
from pyspark.sql.functions import isnan, when, count, col
readings_copy.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in readings_copy.columns]).count()

1

In [9]:
# Check for Class imbalance

readings_copy \
.groupBy(readings_copy["event"]) \
.agg({"experiment":"count"}).alias("knt") \
.show()

+-----+-----------------+
|event|count(experiment)|
+-----+-----------------+
|    B|            92550|
|    D|           184497|
|    C|          1369342|
|    A|          2215328|
+-----+-----------------+



# Multinomial Logistic Regression

In [10]:
numeric_features = [col[0] for col in readings_copy.dtypes if col[1] in ('int','double')]

In [23]:
#readings_copy.select(numeric_features).sample(True, 0.01, 100).count()

In [60]:
##### Reading from pickle file into pandas directly, since toPandas is not working #####
##### numeric_data = readings_copy.select(numeric_features).sample(False, 0.0001, 100).toPandas()

In [11]:
sc._conf.getAll()

[(u'spark.master', u'local'),
 (u'spark.eventLog.dir', u'hdfs://cluster-9126-m/user/spark/eventlog'),
 (u'spark.dynamicAllocation.minExecutors', u'1'),
 (u'spark.executor.memory', u'2688m'),
 (u'spark.driver.host',
  u'cluster-9126-m.us-central1-b.c.eco-plating-240003.internal'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.yarn.historyServer.address', u'cluster-9126-m:18080'),
 (u'spark.history.fs.logDirectory',
  u'hdfs://cluster-9126-m/user/spark/eventlog'),
 (u'spark.yarn.am.memory', u'640m'),
 (u'k1', u'vtest'),
 (u'spark.driver.port', u'37801'),
 (u'spark.executor.instances', u'2'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.executorEnv.PYTHONPATH',
  u'/usr/lib/spark/python/lib/py4j-0.10.7-src.zip:/usr/lib/spark/python/:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.ui.filters',
  u'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 (u'spark.app.name', u'mySparkSession'),
 (u'spark.

In [12]:
#import pandas.plotting as pd
#axs = pd.scatter_matrix(numeric_data, figsize=(8, 8));

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.stat import Statistics
import pandas as pd

#Correlation.corr(numeric_data, "features", "spearman").head()

numeric_data=readings_copy.select(numeric_features).sample(False, 0.0001, 100)
features = numeric_data.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = numeric_features, numeric_features

In [12]:
corr_df

,crew,time,seat,eeg_fp1,eeg_f7,eeg_f8,eeg_t4,eeg_t6,eeg_t5,eeg_t3,...,eeg_f4,eeg_c4,eeg_p4,eeg_poz,eeg_c3,eeg_cz,eeg_o2,ecg,r,gsr
crew,1.000000,0.035681,-0.034516,0.010167,0.029228,0.030929,0.022372,0.041766,0.073071,0.080733,...,-0.008462,0.045842,0.058912,0.016488,0.062108,-0.016019,0.043449,-0.124068,-0.025677,0.105019
time,0.035681,1.000000,-0.042263,-0.010132,-0.018483,-0.017457,0.001697,0.037786,0.031582,-0.012636,...,-0.008335,-0.016064,0.014853,0.006492,0.027272,0.035607,0.042983,0.016632,-0.051065,0.007105
seat,-0.034516,-0.042263,1.000000,-0.029397,-0.032257,0.029459,-0.018450,-0.005687,0.007809,-0.031192,...,0.005658,0.034901,0.014634,-0.050217,-0.081303,0.014257,-0.032967,-0.028155,0.890607,-0.265874
eeg_fp1,0.010167,-0.010132,-0.029397,1.000000,0.838476,0.648212,0.372431,0.267261,0.330635,0.562953,...,0.658064,0.558648,0.445194,0.457723,0.679484,0.501732,0.342321,-0.041246,-0.022333,-0.049776
eeg_f7,0.029228,-0.018483,-0.032257,0.838476,1.000000,0.577236,0.372454,0.298850,0.418026,0.725310,...,0.546586,0.541787,0.455241,0.485452,0.763528,0.455938,0.375053,-0.098638,-0.026256,-0.058926
eeg_f8,0.030929,-0.017457,0.029459,0.648212,0.577236,1.000000,0.621345,0.433249,0.307215,0.447801,...,0.656923,0.609995,0.548873,0.469095,0.551710,0.499765,0.422265,-0.097956,0.039068,-0.034925
eeg_t4,0.022372,0.001697,-0.018450,0.372431,0.372454,0.621345,1.000000,0.425329,0.199515,0.317543,...,0.447077,0.480206,0.454711,0.404575,0.459737,0.358988,0.359415,-0.123762,-0.023871,0.031046
eeg_t6,0.041766,0.037786,-0.005687,0.267261,0.298850,0.433249,0.425329,1.000000,0.364694,0.342008,...,0.334814,0.490523,0.660793,0.573739,0.456737,0.392394,0.627605,-0.108307,0.015296,0.015714
eeg_t5,0.073071,0.031582,0.007809,0.330635,0.418026,0.307215,0.199515,0.364694,1.000000,0.621262,...,0.345515,0.426466,0.543122,0.633985,0.606425,0.429758,0.597367,-0.079759,-0.003693,-0.016306
eeg_t3,0.080733,-0.012636,-0.031192,0.562953,0.725310,0.447801,0.317543,0.342008,0.621262,1.000000,...,0.494144,0.540857,0.528352,0.588225,0.750794,0.474398,0.510203,-0.089446,-0.027610,-0.037997


In [13]:
from pyspark.sql.types import *
from pyspark.sql import Row,Column

In [14]:
out_loop=0
in_loop=0

for col in corr_df.columns:
    for col0 in corr_df.columns:
        if out_loop > in_loop and ((corr_df[col][col0] >= 0.7) or (corr_df[col][col0] <= -0.7)):
            print col, col0, corr_df[col][col0]
        in_loop += 1
    out_loop += 1
    in_loop=0

eeg_f7 eeg_fp1 0.8384760708991459
eeg_t3 eeg_f7 0.7253096041013797
eeg_fp2 eeg_fp1 0.9462699136713915
eeg_fp2 eeg_f7 0.7821287665018913
eeg_fp2 eeg_f8 0.7490962673781135
eeg_p3 eeg_t5 0.742043620073261
eeg_p3 eeg_o1 0.8398427573262578
eeg_fz eeg_fp1 0.7383237135847583
eeg_fz eeg_f7 0.7087558912671861
eeg_fz eeg_fp2 0.7268267543887664
eeg_p4 eeg_p3 0.7285462139890492
eeg_p4 eeg_c4 0.8120593483202032
eeg_poz eeg_o1 0.8327349947750927
eeg_poz eeg_p3 0.8527945101782315
eeg_poz eeg_p4 0.8273102444295671
eeg_c3 eeg_f7 0.7635283760314844
eeg_c3 eeg_t3 0.7507944139548841
eeg_c3 eeg_p3 0.8325102136065194
eeg_c3 eeg_c4 0.7172077013672159
eeg_c3 eeg_poz 0.7423061035085815
eeg_o2 eeg_o1 0.810270339301154
eeg_o2 eeg_p3 0.7444351153862543
eeg_o2 eeg_p4 0.8178544925737878
eeg_o2 eeg_poz 0.849090594317823
r seat 0.8906067586697861


In [15]:
columns_to_drop = ['eeg_fp1','eeg_f8','eeg_o1','eeg_fp2','eeg_p3','eeg_p4','eeg_c3','eeg_cz','eeg_poz','r']
readings_copy_t2=readings_copy.drop(*columns_to_drop)

In [16]:
numeric_features = [col[0] for col in readings_copy_t2.dtypes if col[1] in ('int','double')]
numeric_data=readings_copy_t2.select(numeric_features).sample(False, 0.0001, 100)
features = numeric_data.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = numeric_features, numeric_features

In [17]:
out_loop=0
in_loop=0

for col in corr_df.columns:
    for col0 in corr_df.columns:
        if out_loop > in_loop and ((corr_df[col][col0] >= 0.7) or (corr_df[col][col0] <= -0.7)):
            print col, col0, corr_df[col][col0]
        in_loop += 1
    out_loop += 1
    in_loop=0

eeg_t3 eeg_f7 0.7253096041013795
eeg_fz eeg_f7 0.7087558912671864


In [18]:
##### https://www.youtube.com/watch?v=jEyahxFp3ak
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

categoricalColumns = ['crew', 'experiment', 'seat']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

In [19]:
label_stringIdx = StringIndexer(inputCol = 'event', outputCol = 'label')
stages += [label_stringIdx]

In [20]:
numericCols = ['time','eeg_f7','eeg_t4','eeg_t6','eeg_t5','eeg_t3','eeg_pz','eeg_f3','eeg_fz','eeg_f4','eeg_c4','eeg_o2','ecg','gsr']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [21]:
cols = readings_copy_t2.columns

from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(readings_copy_t2)
readings_copy_t2 = pipelineModel.transform(readings_copy_t2)
selectedCols = ['label', 'features']# + cols
readings_copy_t2 = readings_copy_t2.select(selectedCols)
readings_copy_t2.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [22]:
train, test = readings_copy_t2.randomSplit([0.8, 0.2], seed = 123)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 3109158
Test Dataset Count: 777588


In [24]:
#drop_lst = ['event']
#X_train=train.drop(*drop_lst)
#y_train=train[drop_lst]

In [23]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
ovr = OneVsRest(classifier=lr)
#lrModel = lr.fit(train)
lrModel = ovr.fit(train)

In [24]:
from pyspark.ml import param, Transformer, Model
lrModel.params

[Param(parent=u'OneVsRestModel_4e2ebdaa876922fe779d', name='classifier', doc='base binary classifier'),
 Param(parent=u'OneVsRestModel_4e2ebdaa876922fe779d', name='featuresCol', doc='features column name.'),
 Param(parent=u'OneVsRestModel_4e2ebdaa876922fe779d', name='labelCol', doc='label column name.'),
 Param(parent=u'OneVsRestModel_4e2ebdaa876922fe779d', name='predictionCol', doc='prediction column name.'),
 Param(parent=u'OneVsRestModel_4e2ebdaa876922fe779d', name='weightCol', doc='weight column name. If this is not set or empty, we treat all instance weights as 1.0.')]

In [25]:
#print("Coefficients: \n" + str(lrModel.coefficientMatrix))
#print("Intercept: " + str(lrModel.interceptVector))

#print("Coefficients: \n" + str(lrModel.coefficients))
#print("Intercept: " + str(lrModel.intercept))

In [26]:
ovr.getLabelCol()

'label'

In [39]:
#trainingSummary = lrModel.summary

In [41]:
#### Objective function (scaled loss + regularization) at each iteration.
#trainingSummary.objectiveHistory

In [42]:
#accuracy = trainingSummary.accuracy
#falsePositiveRate = trainingSummary.weightedFalsePositiveRate
#truePositiveRate = trainingSummary.weightedTruePositiveRate
#fMeasure = trainingSummary.weightedFMeasure()
#precision = trainingSummary.weightedPrecision
#recall = trainingSummary.weightedRecall

In [46]:
#type(trainingSummary)

In [27]:
train \
.groupBy(train["label"]) \
.agg({"features":"count"}).alias("knt") \
.show()

+-----+---------------+
|label|count(features)|
+-----+---------------+
|  0.0|        1795943|
|  1.0|        1087611|
|  3.0|          76061|
|  2.0|         149543|
+-----+---------------+



In [28]:
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

False positive rate by label:


NameError: name 'trainingSummary' is not defined

In [81]:
print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

True positive rate by label:
label 0: 0.998656239407
label 1: 1.0
label 2: 1.99302441455e-05
label 3: 0.0


In [82]:
print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

Precision by label:
label 0: 0.888354906118
label 1: 0.997752129392
label 2: 0.428571428571
label 3: 0.0


In [83]:
print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

Recall by label:
label 0: 0.998656239407
label 1: 1.0
label 2: 1.99302441455e-05
label 3: 0.0


In [84]:
print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

F-measure by label:
label 0: 0.940281854621
label 1: 0.998874800044
label 2: 3.98586347089e-05
label 3: 0.0


In [85]:
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.926291391448
FPR: 0.101563290021
TPR: 0.926291391448
F-measure: 0.891983771911
Precision: 0.882047331214
Recall: 0.926291391448


In [30]:
predictions = lrModel.transform(test)

In [31]:
predictions.count()

777588

In [32]:
predictions \
.groupBy(predictions["label"], predictions["prediction"]) \
.agg({"features":"count"}).alias("knt") \
.show()

+-----+----------+---------------+
|label|prediction|count(features)|
+-----+----------+---------------+
|  2.0|       0.0|          37324|
|  1.0|       1.0|         271595|
|  0.0|       1.0|            628|
|  0.0|       0.0|         448845|
|  3.0|       0.0|          19196|
+-----+----------+---------------+



In [33]:
test \
.groupBy(test["label"]) \
.agg({"features":"count"}).alias("knt") \
.show()

+-----+---------------+
|label|count(features)|
+-----+---------------+
|  0.0|         449473|
|  1.0|         271595|
|  3.0|          19196|
|  2.0|          37324|
+-----+---------------+



In [62]:
#predictions.select('label', 'rawPrediction', 'prediction', 'probability').show(10)

In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

accuracy=evaluator.evaluate(predictions)
print('Accuracy' , accuracy)
print("Test Error = %g " , (1.0 - accuracy))

('Accuracy', 0.9265060674804653)
('Test Error = %g ', 0.07349393251953473)


# Naive Bayes (Multinomial)

In [68]:
columns_to_drop = ['eeg_fp1','eeg_f8','eeg_o1','eeg_fp2','eeg_p3','eeg_p4','eeg_c3','eeg_cz','eeg_poz','r']
readings_copy_t2=readings_copy.drop(*columns_to_drop)

In [69]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, VectorIndexer, MinMaxScaler

categoricalColumns = ['crew', 'experiment', 'seat']
stages = []

for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]


In [70]:
label_stringIdx = StringIndexer(inputCol = 'event', outputCol = 'label')
stages += [label_stringIdx]

In [71]:
numericCols = ['time','eeg_f7','eeg_t4','eeg_t6','eeg_t5','eeg_t3','eeg_pz','eeg_f3','eeg_fz','eeg_f4','eeg_c4','eeg_o2','ecg','gsr']

assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="Features")
stages += [assembler]

In [72]:
scaler = MinMaxScaler(inputCol="Features", outputCol="features")
stages += [scaler]

In [73]:
cols = readings_copy_t2.columns

from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(readings_copy_t2)
readings_copy_t2 = pipelineModel.transform(readings_copy_t2)
selectedCols = ['label', 'features']# + cols
readings_copy_t2 = readings_copy_t2.select(selectedCols)
readings_copy_t2.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [74]:
train, test = readings_copy_t2.randomSplit([0.8, 0.2], seed = 123)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 3109158
Test Dataset Count: 777588


In [75]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [76]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)
predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[0.0,0.0,0.0,0.0,...|[-19.482156856955...|[0.63354663246435...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-18.502357193791...|[0.63023699191442...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-19.172466626749...|[0.63179426098502...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-18.977629309902...|[0.63027083945886...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-14.183699899425...|[0.61469462559990...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-22.064009976469...|[0.64095335527195...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-18.434169318079...|[0.62954273530283...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-16.818309245074...|[0.62516753480343...|       0.0|
|  0.0|[0.0,0.0,0.0,0.0,...|[-21.664818190498...|[0.64023544648534...|       0.0|
|  0.0|[0.0,0.0,

In [78]:
predictions \
.groupBy(predictions["label"], predictions["prediction"]) \
.agg({"features":"count"}).alias("knt") \
.show()

+-----+----------+---------------+
|label|prediction|count(features)|
+-----+----------+---------------+
|  2.0|       0.0|          37324|
|  1.0|       1.0|         271595|
|  0.0|       1.0|            577|
|  0.0|       0.0|         448896|
|  3.0|       0.0|          19196|
+-----+----------+---------------+



In [77]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.926571654912


# XGBoost Classification

In [ ]:
#https://towardsdatascience.com/pyspark-and-xgboost-integration-tested-on-the-kaggle-titanic-dataset-4e75a568bdb